<a href="https://colab.research.google.com/github/ua-deti-information-retrieval/Neural-IR-hands-on/blob/main/RI_practical_tutorial_5_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers ranx accelerate
!git clone https://github.com/ua-deti-information-retrieval/Neural-IR-hands-on.git



In [3]:
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EVCkcYaBlo9GhK0pbIS3kbwBbni5AodWh8Br9q1sg4oWKA?e=nROIqj&download=1" -O dev_set_nq_bm25.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EeG_M5lDSNdKonIxIe7Op_8B-bfkupPZoxk7dRICJ8pS4Q?e=ffBdWy&download=1" -O dev_set_nq_gs.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EYkZF-SVwQlEo-2JShc5cvMBSDB7osa3jM_-hrR86_XJnA?e=hTYh3W&download=1" -O train_set_nq_neg.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EVNRdNHufJJCklmhU2VY3zgBkWsf4fO2mbamfGf2qZ3GRw?e=wfe3fj&download=1" -O train_set_nq_pos.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EY2sTtQg_QlHjkhYeE2fmFUBN4KJnmw2DSdJZGorEbMQhA?e=JqmpH2&download=1" -O collection_with_texts.jsonl

--2023-12-05 22:22:02--  https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EUu_IXNpMqVOrllJDas5kEIBkzfbU-s5KXG4bAFDAFRsrQ?e=W0NOEB&download=1
Resolving uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/aleixomatos_ua_pt/Documents/@UA/Aulas/RI/2023/Dossier/Data/dataset_for_demo/syn_neg_gpt-neo-1.3B_BS_5_k100.jsonl?ga=1 [following]
--2023-12-05 22:22:03--  https://uapt33090-my.sharepoint.com/personal/aleixomatos_ua_pt/Documents/@UA/Aulas/RI/2023/Dossier/Data/dataset_for_demo/syn_neg_gpt-neo-1.3B_BS_5_k100.jsonl?ga=1
Reusing existing connection to uapt33090-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 667711873 (637M) [application/octet-stream]
Saving to: ‘negatives.jsonl’

negatives.jsonl     100%[=====

--2023-12-06 00:00:01--  https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EY2sTtQg_QlHjkhYeE2fmFUBN4KJnmw2DSdJZGorEbMQhA?e=JqmpH2&download=1
Resolving uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/aleixomatos_ua_pt/Documents/@UA/Aulas/RI/2023/Dossier/Data/dataset_for_demo/collection_with_texts.jsonl?ga=1 [following]
--2023-12-06 00:00:02--  https://uapt33090-my.sharepoint.com/personal/aleixomatos_ua_pt/Documents/@UA/Aulas/RI/2023/Dossier/Data/dataset_for_demo/collection_with_texts.jsonl?ga=1
Reusing existing connection to uapt33090-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 1419795157 (1.3G) [application/octet-stream]
Saving to: ‘collection_with_texts.jsonl’

collection_with_tex 100%[==========

# Starting here

In [1]:
!cd Neural-IR-hands-on && git pull

!cp Neural-IR-hands-on/trainer/* .

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 376 bytes | 376.00 KiB/s, done.
From https://github.com/ua-deti-information-retrieval/Neural-IR-hands-on
   72ce7ad..daeea68  main       -> origin/main
Updating 72ce7ad..daeea68
Fast-forward
 trainer/data.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)


In [10]:
from data import get_qrels, InferenceRankingIterator,  BioASQPointwiseIterator, InferenceDataset, create_training_dataset
from sampler import BasicSampler
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

train_ds = create_training_dataset("train_set_nq_pos.jsonl",
                                   "train_set_nq_neg.jsonl",
                                   "collection_with_texts.jsonl",
                                    tokenizer=tokenizer,
                                    iterator_class=BioASQPointwiseIterator[BasicSampler],
                                    max_questions=10000,
                                            )

dev_ds = InferenceDataset("dev_set_nq_bm25.jsonl",
                          train_ds.collection,
                          tokenizer,
                          max_questions=10,
                          at=100, #max docs
                          gs_path="dev_set_nq_gs.jsonl",
                          iterator_class=InferenceRankingIterator)


In [11]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "IRRELEVANT", 1: "RELEVANT"}
label2id = {"IRRELEVANT": 0, "RELEVANT": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
next(iter(dev_ds))

{'input_ids': [101, 2339, 2079, 2417, 2668, 4442, 2444, 2005, 2069, 6036, 2420, 102, 2417, 2668, 3526, 2529, 2417, 2668, 4442, 2024, 2550, 2083, 1037, 2832, 2315, 9413, 22123, 8093, 7361, 10448, 19009, 1010, 4975, 2013, 5462, 7872, 4442, 2000, 9677, 2417, 2668, 4442, 1999, 2055, 1021, 2420, 1012, 2043, 9677, 2094, 1010, 1999, 1037, 7965, 3265, 2122, 4442, 2444, 1999, 2668, 9141, 2005, 2055, 2531, 2000, 6036, 2420, 1006, 1998, 3770, 2000, 3938, 2420, 1999, 1037, 2440, 2744, 10527, 1007, 1012, 1031, 3429, 1033, 2012, 1996, 2203, 1997, 2037, 26462, 1010, 2027, 2024, 3718, 2013, 9141, 1012, 1999, 2116, 11888, 7870, 1010, 1996, 26462, 1997, 1996, 2417, 2668, 4442, 2003, 4359, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=5,
                                  learning_rate=2e-5,
                                  weight_decay=0.01,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=64,
                                  evaluation_strategy= "epoch",
                                  dataloader_pin_memory=True,
                                  output_dir="trained_model",
                                  logging_strategy="steps",
                                  logging_first_step=True,
                                  logging_steps=100,
                                  save_strategy="epoch",
                                  save_total_limit=2,
                                  seed=42)

In [18]:
from ranker_trainer import RankerTrainer
from transformers import DataCollatorWithPadding
from collator import RankingCollator
from metrics import RanxMetrics

import torch
trainer = RankerTrainer(
      model=model,
      args=training_args,
      train_dataset=train_ds,
      eval_dataset=dev_ds,
      tokenizer=tokenizer,
      data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
      eval_data_collator=RankingCollator(tokenizer=tokenizer),
      preprocess_logits_for_metrics=lambda logits, labels: torch.nn.functional.softmax(logits, dim=-1)[:,1], #
      compute_metrics=RanxMetrics(dev_ds.get_qrels()),
  )

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss


Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: ignored

# Use a trained model to run inference!

In [20]:
model = AutoModelForSequenceClassification.from_pretrained("trained_model/checkpoint-6")